# Processing, Nettoyage & Modélisation

Variable cible : **`log_prevalence`** (taux normalisé par région = `Ntop / Npop`)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss

df = pd.read_csv('effectifs.csv', sep=';')

## 1. Nettoyage des données

On supprime :
- Les lignes où `Ntop` est manquant (nécessaire pour calculer la prévalence)
- La région 99 (France entière) qui biaiserait l'analyse régionale

In [ ]:
# Suppression des lignes où Ntop est manquant
df = df.dropna(subset=['Ntop'])

# Suppression de la région 99 (France entière)
df.drop(df[df['region'] == 99].index, inplace=True)

print('Shape après nettoyage :', df.shape)
print('Valeurs manquantes sur Ntop :', df['Ntop'].isnull().sum())
print('Régions présentes :', sorted(df['region'].unique()))

## 2. Agrégation et recalcul de la prévalence

On agrège par `(annee, region, patho_niv1)` et on recalcule :

> `prevalence = ΣNtop / ΣNpop`

La colonne `prev` originale n'est **pas utilisée** car :
- Calculée à un niveau trop fin (département + âge + sexe)
- Non comparable entre régions
- Contient trop de NaN

In [ ]:
df_model = (
    df.groupby(['annee', 'region', 'patho_niv1'])
    .agg({'Ntop': 'sum', 'Npop': 'sum'})
    .reset_index()
)

df_model['prevalence'] = df_model['Ntop'] / df_model['Npop']

print('Shape df_model :', df_model.shape)
print('NaN dans prevalence :', df_model['prevalence'].isnull().sum())
df_model.head()

## 3. Transformation logarithmique de la variable cible

La prévalence est asymétrique (skewness ≈ 2.77 d'après l'EDA).

On applique `log1p(prevalence)` pour :
- Corriger l'asymétrie
- Réduire l'impact des outliers
- Stabiliser les prédictions

`log1p(x) = log(1 + x)` fonctionne même sur les valeurs proches de 0

In [ ]:
df_model['log_prevalence'] = np.log1p(df_model['prevalence'])

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(df_model['prevalence'], bins=50, color='steelblue')
axes[0].set_title('Distribution prevalence (originale)')

axes[1].hist(df_model['log_prevalence'], bins=50, color='green')
axes[1].set_title('Distribution log_prevalence (après transformation)')

plt.tight_layout()
plt.show()

print(f"Skewness avant : {df_model['prevalence'].skew():.2f}")
print(f"Skewness après : {df_model['log_prevalence'].skew():.2f}")

## 4. Définition des features et split train/test

**Features retenues :** `annee`, `patho_niv1`, `region`

**Variable cible :** `log_prevalence`

**Variables exclues :**
- `prev` originale : trop de NaN, niveau trop fin
- `Npop` : data leakage potentiel
- `Ntop` : utilisé pour calculer la cible

**Split 80/20** avec `random_state=42`

In [ ]:
from sklearn.model_selection import train_test_split

features = ['annee', 'patho_niv1', 'region']
target   = 'log_prevalence'

X = df_model[features]
y = df_model[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print('X_train :', X_train.shape)
print('X_test  :', X_test.shape)

## 5. Préprocesseur & Pipeline

- `annee`, `region` → pipeline **numérique** (imputer médiane + StandardScaler)
- `patho_niv1` → pipeline **catégoriel** (imputer mode + OrdinalEncoder)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

numeric_features     = ['annee', 'region']
categorical_features = ['patho_niv1']

numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler',  StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder())
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline,     numeric_features),
    ('cat', categorical_pipeline, categorical_features)
])

print(preprocessor)

## 6. Modèle 1 — Régression Linéaire

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import joblib

lin_pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('model', LinearRegression())
])

lin_pipeline.fit(X_train, y_train)
y_pred_lin = lin_pipeline.predict(X_test)

rmse_lin = np.sqrt(mean_squared_error(y_test, y_pred_lin))
r2_lin   = r2_score(y_test, y_pred_lin)

print('Linear Regression')
print('RMSE:', rmse_lin)
print('R2  :', r2_lin)

joblib.dump(lin_pipeline, '../model/linear.pkl')

## 7. Modèle 2 — Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('model', RandomForestRegressor(n_estimators=300, random_state=42))
])

rf_pipeline.fit(X_train, y_train)
y_pred_rf = rf_pipeline.predict(X_test)

rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf   = r2_score(y_test, y_pred_rf)

print('Random Forest')
print('RMSE:', rmse_rf)
print('R2  :', r2_rf)

joblib.dump(rf_pipeline, '../model/random_forest.pkl')

## 8. Modèle 3 — Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gb_pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('model', GradientBoostingRegressor(random_state=42))
])

gb_pipeline.fit(X_train, y_train)
y_pred_gb = gb_pipeline.predict(X_test)

rmse_gb = np.sqrt(mean_squared_error(y_test, y_pred_gb))
r2_gb   = r2_score(y_test, y_pred_gb)

print('Gradient Boosting')
print('RMSE:', rmse_gb)
print('R2  :', r2_gb)

joblib.dump(gb_pipeline, '../model/gradient_boosting.pkl')

## 9. Comparaison des 3 modèles

In [ ]:
results = pd.DataFrame({
    'Modèle' : ['Linear Regression', 'Random Forest', 'Gradient Boosting'],
    'RMSE'   : [rmse_lin, rmse_rf, rmse_gb],
    'R2'     : [r2_lin,   r2_rf,   r2_gb]
})

print(results.to_string(index=False))

## 10. Test de data leakage

On mélange la target d'entraînement et on vérifie que le R2 chute proche de 0.

Si le R2 reste élevé → data leakage détecté.

In [ ]:
y_train_shuffled = np.random.permutation(y_train)

rf_pipeline.fit(X_train, y_train_shuffled)
y_pred_shuffled = rf_pipeline.predict(X_test)

r2_leak_test = r2_score(y_test, y_pred_shuffled)
print('R2 avec target mélangée :', r2_leak_test)
print('-> Pas de data leakage' if r2_leak_test < 0.1 else '⚠️ Data leakage potentiel détecté')